In [6]:
# Import libraries
import numpy as np
import pandas as pd
import datetime as dt
import os
import pickle
import pymc3 as pm

# SetUp the path
path = os.chdir('/Users/mac/Documents/Voodoo/ADN-DS')

# Load_data
df_all = pd.read_csv('modeling_ipm_data_v2.csv', index_col = False)

# Get last 7 days from dataset (embedding includet into last date)
df_all = df_all[pd.to_datetime(df_all['date']) == pd.to_datetime(df_all.date.max())]

# Group data and calculate 'total_impressions' and 'total_install'
def Total_Data_IPM(df, features, group_by_feature, sort_features):    

        # Create dataset for AD Selection task
        df = df[features]

        df_grouped = df.groupby(group_by_feature)['window_impressions', 'window_installs'].sum()

        df_grouped.rename(columns = {'window_impressions':'total_impressions', 'window_installs':'total_install'}, inplace = True)

        df = df_grouped.reset_index()

        # Sort values
        df.sort_values(sort_features)

        return df
    
def train_model(df_all):
   
    # Grouping data for model
    df_group = Total_Data_IPM(df_all, 
                             ['promoted_app', 'ad_id', 'window_impressions','window_installs'], 
                             ['promoted_app', "ad_id"],
                              'promoted_app')
    
    # Set variables
    n = df_group['total_impressions'].values
    k = df_group['total_install'].values

    # Build model for each day
    with pm.Model() as model:

        # Set priors
        alpha = pm.Uniform('alpha', 1e-3, 1e5)
        beta = pm.Uniform('beta',  1e-3, 1e5)

        p = pm.Beta('p', alpha = alpha, beta = beta, shape = len(n))

        # Specify model
        obs = pm.Binomial('obs', n = n, p = p, observed = k) # (C_n)^k * p^k * (1-p)^(n-k) 

        # Configure sampler.
        trace = pm.sample(
                        #draws = int(1e4), # The number of samples to draw (1e4)
                        #tune = int(5e3), # Number of iterations to tune (5e3)
                        target_accept = 0.9, #  The step size is tuned such that we approximate this acceptance rate. Higher values like 0.9 or 0.95 often work better for problematic posteriors. 
                        return_inferencedata = True, # return the trace as an arviz.InferenceData (True) object
                        idata_kwargs = {'log_likelihood': False},
                        random_seed = 42) 
    
    # Save model
    with open('Model/ad_selection_model.pkl', 'wb') as buff:
        pickle.dump({'model': model, 'trace': trace}, buff)

In [7]:
train_model(df_all)

/var/folders/57/nx0ld9vx17l0yw9shbwxhk440000gn/T/ipykernel_1174/3783839664.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_grouped = df.groupby(group_by_feature)['window_impressions', 'window_installs'].sum()
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p, beta, alpha]


/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 193 seconds.
There were 805 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5560800464733382, but should be close to 0.9. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 837 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7288551068352215, but should be close to 0.9. Try to increase the number of tuning steps.
There were 424 divergences after tuning. Increase `target_accept`